In [1]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l


In [2]:
max_degree = 20
n_train, n_test = 100, 100
true_w = np.zeros(max_degree)
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])
features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)


In [3]:
true_w, features, poly_features, labels = [
    torch.tensor(x, dtype=torch.float32) for x in [true_w, features, poly_features, labels]
]
for x in [true_w, features, poly_features, labels]:
    x = torch.tensor(x, dtype=torch.float32)

features[:2], poly_features[:2, :], labels[:2]


/var/folders/n5/6rbsb_g91xvbk54r7z6gv_jm0000gn/T/ipykernel_19629/3095538953.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


(tensor([[-0.3402],
         [-0.0690]]),
 tensor([[ 1.0000e+00, -3.4018e-01,  5.7861e-02, -6.5611e-03,  5.5799e-04,
          -3.7963e-05,  2.1524e-06, -1.0460e-07,  4.4478e-09, -1.6812e-10,
           5.7190e-12, -1.7686e-13,  5.0138e-15, -1.3120e-16,  3.1880e-18,
          -7.2299e-20,  1.5372e-21, -3.0759e-23,  5.8132e-25, -1.0408e-26],
         [ 1.0000e+00, -6.9028e-02,  2.3824e-03, -5.4818e-05,  9.4598e-07,
          -1.3060e-08,  1.5025e-10, -1.4816e-12,  1.2784e-14, -9.8051e-17,
           6.7682e-19, -4.2472e-21,  2.4431e-23, -1.2973e-25,  6.3962e-28,
          -2.9434e-30,  1.2699e-32, -5.1562e-35,  1.9774e-37, -7.1838e-40]]),
 tensor([4.3425, 5.0414]))

In [4]:
def evaluate_loss(net, data_iter, loss):
    """评估给定数据集上模型的损失。"""
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]


In [6]:
def train(train_features, test_features, train_labels, test_labels, num_epochs=400):
    loss = nn.MSELoss()
    input_shape = train_features.shape[-1]
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(-1, 1)), batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(1, -1)), batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', xlim=[1, num_epochs], ylim=[1e-3, 1])
    
    for epoch in range(num_epochs):
        train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss), evaluate_loss(net, test_iter, loss)))
        print('weight:', net[0].weight.data.numpy())


In [7]:
train(poly_features[:n_train, :4], poly_features[n_train:, :4], labels[:n_train], labels[n_train:])


AssertionError: Size mismatch between tensors